In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from PIL import Image
import numpy as np
import pandas as pd

In [22]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=10),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=7),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 128, kernel_size=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=4),
            nn.ReLU(inplace=True),
        )

        self.fc = nn.Sequential(
            nn.Linear(256 * 3 * 3, 4096),
            nn.Sigmoid()
        )

    def forward(self, x1, x2):
        output1 = self.cnn(x1)
        output2 = self.cnn(x2)

        output1 = output1.view(output1.size(0), -1)
        output2 = output2.view(output2.size(0), -1)

        output1 = self.fc(output1)
        output2 = self.fc(output2)

        return output1, output2

In [19]:
import torchvision.transforms as transforms

class SiameseDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __getitem__(self, index):
        image_path = self.data['Filename'].iloc[index]
        brand_name = self.data['Brand Name'].iloc[index]
        label = self.data['Label'].iloc[index]

        image = Image.open(image_path).convert('RGB')

        # Resize the image to a consistent size
        resize = transforms.Resize((70, 70))
        image = resize(image)

        if self.transform is not None:
            image = self.transform(image)

        return image, brand_name, label

    def __len__(self):
        return len(self.data)

In [20]:
def train_siamese_network(siamese_net, train_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs1, inputs2, labels = data
            optimizer.zero_grad()
            outputs1, outputs2 = siamese_net(inputs1, inputs2)
            loss = criterion(outputs1, outputs2, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if i % 10 == 9:
                print('[Epoch: %d, Batch: %5d] Loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
                running_loss = 0.0

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim=True)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [23]:
csv_file = r'C:\Users\ADE17\Desktop\Masters\Projects\LogoGuard--Deep-Learning-for-Fake-Logo-Recognition\file_mapping.csv'
transform = ToTensor()
epochs = 10
batch_size = 32
learning_rate = 0.001

# Create the Siamese dataset
siamese_dataset = SiameseDataset(csv_file, transform=transform)

# Create the Siamese dataloader
train_loader = DataLoader(siamese_dataset, shuffle=True, batch_size=batch_size)

# Initialize the Siamese network
siamese_net = SiameseNetwork()

# Define the contrastive loss function and optimizer
criterion = ContrastiveLoss()
optimizer = optim.Adam(siamese_net.parameters(), lr=learning_rate)

# Train the Siamese network
train_siamese_network(siamese_net, train_loader, criterion, optimizer, epochs)

TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
